<a href="https://colab.research.google.com/github/nxxk23/AI-Engineer/blob/main/extract/gradioprompt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install easyocr gradio pythainlp langchain langchain_huggingface langchain_community
!sudo apt-get install ghostscript

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os

resume_directory = '/content/drive/MyDrive/AIEngineer/resume/resume_LLM'
pdf_files = [os.path.join(resume_directory, f) for f in os.listdir(resume_directory) if f.endswith('.pdf')]

# Print the list of PDF files to verify
print(pdf_files)

['/content/drive/MyDrive/AIEngineer/resume/resume_LLM/Resume BA 2.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/Resume BA 3.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/Resume ba 5.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/Resume ba2.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/Resume baa.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/resume 6.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/resume 7.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/resume 2.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/resume 3.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/resume 4.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/resume 5.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/resume 1.pdf']


## **Job Qualification**

| **ᴊᴏʙ** | **ᴘᴏꜱɪᴛɪᴏɴ** | **ᴄᴏᴜɴᴛ** |
|:---:|:---:|:---:|
| 1 | ʙᴜꜱɪɴᴇꜱꜱ ᴀɴᴀʟʏꜱᴛ | 5 |
| 2 | ᴀɪ ᴇɴɢɪɴᴇᴇʀ | 7 |


In [7]:
import os
import re
import numpy as np
import pandas as pd
import easyocr
import subprocess
from pythainlp.phayathaibert.core import NamedEntityTagger
import gradio as gr
from PIL import Image
import requests
from threading import Semaphore
from concurrent.futures import ThreadPoolExecutor, as_completed
from huggingface_hub import InferenceClient

In [16]:
resume_prompt = """
You are a highly experienced Human Resource Recruiter known for your unwavering commitment to excellence.
At your company, only the most exceptional candidates are considered, and you scrutinize each resume with a discerning eye.
You are tasked with evaluating this candidate against the detailed job description provided.

### **Instructions:**
As you review the resume below for a [Position], consider whether this candidate truly differentiates themselves in a highly competitive field.
Rigorously assess their qualifications, skills, and experience in direct comparison to the specific requirements outlined in the job description.

### **Job Description:**
{jobdescribe}

{resume}

### **Evaluation Template:**
Classification: [Pass / Borderline / Not Pass]
Explanation: [Identify the explanation of the candidate's classification]

### **Classification Criteria:**
- **Pass**: The candidate distinctly exceeds the job requirements, displaying exceptional qualifications and achievements that make them an ideal fit for the role.
- **Borderline**: The candidate is adequate, meeting some key qualifications but lacking in others. They are neither remarkable nor entirely unqualified and may need further consideration.
- **Not Pass**: The candidate lacks critical qualifications or experience and does not meet the key requirements for the position.
"""


In [17]:
# Function to convert PDF to images using Ghostscript
def convert_pdf_to_images(pdf_file, output_folder="/content/images", dpi=300):
    try:
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
        output_format = os.path.join(output_folder, "page_%03d.png")
        gs_command = [
            "gs",
            "-sDEVICE=png16m",
            f"-r{dpi}",
            "-o", output_format,
            pdf_file
        ]
        subprocess.run(gs_command, check=True)
        image_files = sorted([os.path.join(output_folder, f) for f in os.listdir(output_folder) if f.endswith(".png")])
        return image_files
    except subprocess.CalledProcessError as e:
        print(f"Ghostscript error: {e}")
        return []

# Function to extract text from a single image
def extract_text_from_image(image_path):
    reader = easyocr.Reader(['th', 'en'], gpu=True)
    image = Image.open(image_path)
    image_np = np.array(image)
    result = reader.readtext(image_np)
    sorted_data = sorted(result, key=lambda x: x[0][0][1])
    plain_text = "\n".join([text for _, text, _ in sorted_data])
    return plain_text

# Function to split text into chunks based on token length
def chunk_text(text, tokenizer, max_tokens=510):
    tokens = tokenizer.tokenize(text)
    chunks = []
    for i in range(0, len(tokens), max_tokens):
        chunk = tokenizer.convert_tokens_to_string(tokens[i:i+max_tokens])
        chunks.append(chunk)
    return chunks

# Tag and clean text with chunking
def tag_and_clean_text(text, tagger, tokenizer, unwanted_pattern):
    text_chunks = chunk_text(text, tokenizer)

    tagged_text = ""
    cleaned_text = ""

    for chunk in text_chunks:
        try:
            ner = tagger.get_ner(chunk, tag=True)
            pattern = r'<(?!ORGANIZATION|PERCENT|TIME)[^>]+>[^<]+</[^>]+>'
            cleaned_ner = re.sub(pattern, '', ner)
            cleaned_ner = re.sub(r'</?(ORGANIZATION|PERCENT|TIME)>', '', cleaned_ner)
            cleaned_ner = re.sub(unwanted_pattern, '', cleaned_ner, flags=re.IGNORECASE)
            cleaned_ner = re.sub(r'\bal\b', 'ai', cleaned_ner, flags=re.IGNORECASE)

            tagged_text += ner + "\n"
            cleaned_text += cleaned_ner + "\n"
        except Exception as e:
            print(f"Error processing chunk: {e}")

    return tagged_text.strip(), cleaned_text.strip()

# Function to generate an answer using the prompt
def generate_answer(resume, job):
    formatted_prompt = resume_prompt.replace("{resume}", resume).replace("{jobdescribe}", job)
    client = InferenceClient('https://ai-api.manageai.co.th/llm-model-02/')
    response = client.text_generation(formatted_prompt, **model_params)
    output = "".join(response)
    return output

# Function to extract classification from the evaluation text
def extract_classification(text):
    match = re.search(r'Classification:\s*(Pass|Borderline|Not Pass)', text)
    if match:
        return match.group(1)
    return text

# Function to process a single PDF
def process_single_pdf(pdf_file, job):
    results = []
    try:
        images = convert_pdf_to_images(pdf_file.name)
        raw_text = ""
        for image_path in images:
            raw_text += extract_text_from_image(image_path) + "\n"

        tagger = NamedEntityTagger()
        tokenizer = tagger.tokenizer

        unwanted_terms = [
            'ที่อยู่', 'โทรศัพท์', 'อีเมล', 'linkedin', ':', ',', '-', '|',
            'ประวัติส่วนตัว', 'เกี่ยวกับฉัน', 'about me', 'ชื่อ', 'สกุล', 'tell', 'โทร', 'โทรงาน',
            'ชื่อเล่น', 'อายุ', 'วันเกิด', 'พุทธ', 'ศาสนา', 'สัญชาติ', 'phone',
            'ช่องทางการติดต่อ', '_', 're sume', 'resume', 'resu me'
        ]
        unwanted_pattern = '|'.join(map(re.escape, unwanted_terms))

        tagged_text, cleaned_text = tag_and_clean_text(raw_text, tagger, tokenizer, unwanted_pattern)
        evaluation = generate_answer(cleaned_text, job)
        result = extract_classification(evaluation)

        results.append({
            "PDF File": os.path.basename(pdf_file.name),
            "Raw_Text": raw_text,
            "Tagged_Text": tagged_text,
            "Cleaned_Text": cleaned_text,
            "Evaluation": evaluation,
            "Result": result
        })

    except Exception as e:
        print(f"Error in process_single_pdf: {e}")

    df = pd.DataFrame(results)
    return df

# Function to process multiple PDFs
def process_multiple_pdfs(pdf_files, job):
    all_results = pd.DataFrame()
    for pdf_file in pdf_files:
        df = process_single_pdf(pdf_file, job)
        all_results = pd.concat([all_results, df], ignore_index=True)
    return all_results

# Function to save DataFrame to CSV
def save_dataframe(df):
    try:
        csv_path = '/content/output.csv'
        df.to_csv(csv_path, index=False, encoding='utf-8-sig')
        return csv_path
    except Exception as e:
        print(f"Error saving DataFrame: {e}")
        return ""

# Semaphore to limit concurrent API requests
semaphore = Semaphore(50)

# Define the model parameters
model_params = {
    "max_new_tokens": 200,
    "temperature": 0.1,
    "top_p": 0.95,
    "repetition_penalty": 1.0
}

# Function to process a batch of DataFrame rows
def generate_prompt_answer_optimized(row):
    resume = row.get('Cleaned_Text', "")
    job = row.get('Job_Description', "")
    result = generate_answer(resume, job)
    return result

# Function to process a batch of rows
def process_batch(batch_df):
    return [generate_prompt_answer_optimized(row) for _, row in batch_df.iterrows()]

# Batch processing with threading
def batch_process(df, batch_size=32):
    results = [None] * len(df)
    with ThreadPoolExecutor() as executor:
        futures = {}
        for i in range(0, len(df), batch_size):
            batch_df = df.iloc[i:i + batch_size]
            future = executor.submit(process_batch, batch_df)
            futures[future] = (i, i + batch_size)

        for future in as_completed(futures):
            start_idx, end_idx = futures[future]
            batch_results = future.result()
            results[start_idx:end_idx] = batch_results

    return results

# Gradio Interface Function
def gradio_interface(pdf_files, job, max_new_tokens, temperature, top_p, repetition_penalty):
    try:
        # Update model parameters based on user input
        model_params.update({
            "max_new_tokens": max_new_tokens,
            "temperature": temperature,
            "top_p": top_p,
            "repetition_penalty": repetition_penalty
        })

        df = process_multiple_pdfs(pdf_files, job)
        df['Evaluation'] = batch_process(df, batch_size=32)
        df['Result'] = df['Evaluation'].apply(extract_classification)
        csv_path = save_dataframe(df)
        return df, csv_path
    except Exception as e:
        print(f"Error in gradio_interface: {e}")
        return pd.DataFrame(), ""

# Define the Gradio interface
def create_gradio_interface():
    with gr.Blocks() as demo:
        with gr.Row():
            with gr.Column(scale=1):
                pdf_file_input = gr.Files(label="Upload PDF Files", file_count="multiple")
                job_input = gr.Textbox(placeholder="Enter the job description here...", label="Job Description", lines=10)

                # Model parameters
                max_new_tokens_input = gr.Slider(label="Max New Tokens", minimum=50, maximum=1024, value=512, step=50)
                temperature_input = gr.Slider(label="Temperature", minimum=0.01, maximum=1.0, value=0.7, step=0.1)
                top_p_input = gr.Slider(label="Top P", minimum=0.1, maximum=1.0, value=0.95, step=0.05)
                repetition_penalty_input = gr.Slider(label="Repetition Penalty", minimum=0.1, maximum=2.0, value=1.0, step=0.1)

                submit_btn = gr.Button("Process")

            with gr.Column(scale=2):
                output_df = gr.DataFrame(headers=["PDF File", "Raw_Text", "Tagged_Text", "Cleaned_Text", "Evaluation", "Result"], type="pandas", interactive=True)
                output_csv = gr.File(label="Download CSV")

            # Function to process inputs and generate output
            def on_submit(pdf_files, job, max_new_tokens, temperature, top_p, repetition_penalty):
                df, csv_path = gradio_interface(pdf_files, job, max_new_tokens, temperature, top_p, repetition_penalty)
                return df, csv_path

            submit_btn.click(on_submit,
                             inputs=[pdf_file_input, job_input, max_new_tokens_input, temperature_input, top_p_input, repetition_penalty_input],
                             outputs=[output_df, output_csv])

    return demo

In [18]:
# Create and launch the Gradio interface
app = create_gradio_interface()
app.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://48f18c46b36ea5b5fa.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/easyocr/detection.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(copyStateDict(torch.load(trained_model, ma

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://48f18c46b36ea5b5fa.gradio.live


In [19]:
import pandas as pd
AI = pd.read_csv('/content/output.csv')
AI

,PDF File,Raw_Text,Tagged_Text,Cleaned_Text,Evaluation,Result
0,Resume BA 2.pdf,"พิมพ์นารา วัฒนชัย\nที่อยู่: 789 ถนนรัชดาภิเษก,...",<PERSON>พ</PERSON><PERSON>พิมพ์</PERSON><PERSO...,st เป้าหมายในการทํางาน นําความเชี่ยวชาญด้านการ...,\n### **Candidate's Resume:**\n[Insert the c...,Pass
1,Resume ba 5.pdf,081-456-7890\nนัทธมน ฺรัตนโชติ\n nattamon. rad...,<PHONE>08</PHONE><PERSON>1-4</PERSON><PERSON>5...,เป้าหมายในการ มุ่งมันในการนําทักษะด้านการจัดกา...,\n### **Candidate's Resume:**\n\n**Name:** [...,Pass
2,Resume BA 3.pdf,จิตรลดา\n086-234-5678\njitlada.s@domain. com\...,<PERSON>จ</PERSON><PERSON>จิตร</PERSON><PERSON...,เสริม business analyst เป้าหมายในการทํางาน ต้อ...,\n### **Candidate's Resume:**\n\nชื่อ: น.ส. ...,Pass
3,Resume ba2.pdf,ศุภกิจฺ มงคลชัย\n089-234-5678\n supakit .m@do...,<PERSON>ศ</PERSON><PERSON>ศุภ</PERSON><PERSON>...,ประสบการณ์การ เป้าหมายในการทํางาน flnancial an...,\n### **Candidate's Resume:**\n\nชื่อ: สมชาย...,\n### **Candidate's Resume:**\n\nชื่อ: สมชาย...
4,Resume baa.pdf,"ธนกร อินทรีย์พงษ์\n ที่อยู่:\n456 ถนนพระราม 3,...",<PERSON>ธ</PERSON><PERSON>ธน</PERSON><PERSON>ก...,เป้าหมายในการทํางาน มุ่งมันที่จะใช้ความรู้และท...,\n### **Candidate's Resume:**\n**Name:** [Ca...,Pass


In [24]:
print(AI['Evaluation'].iloc[3])

  
### **Candidate's Resume:**

ชื่อ: สมชาย สมศรี  
ที่อยู่: 123 ถนนสุขุมวิท แขวงคลองเตย เขตคลองเตย กรุงเทพฯ 10110  
โทรศัพท์: 081-123-4567  
อีเมล: somchai.somsri@gmail.com  

**ประสบการณ์การทำงาน:**

1. นักวิเคราะห์การเงิน, บริษัท โกลบอล ไฟแนนซ์ จำกัด, 2018 - 2022  
- วิเคราะห์และจัดทํารายงานทางการเงินเพื่อสนับสนุนการตัดสินใจขององค์กร
- ประเมินความเสี่ยงและโอกาสทางการเงินของโครงการลงทุนต่างๆ
- ทํางานร่วมกับทีมบัญชีในการจัดการเงินและวิเคราะห์ผลการดําเนินงาน
- ทักษะ accountant บริษัท global finance financial sql การวิเคราะห์ทางการเงิน excel modeling

2. นักบัญชี, บริษัท ฟินเซิร์ฟ จำกัด, 2016 - 2018  
- จัดทําและตรวจสอบเอกสารทางการเงิน
- การวิเคราะห์ธุรกิจ swot pestel cap analysis
- บันทึกบัญชีในระบบ
- การใช้เครื่องมือ sap. quickbooks power bi

**ทักษะ:**
- ทักษะการวิเคราะห์ทางการเงิน
- ทักษะการวิเคราะห์ธุรกิจ
- ทักษะการใช้เครื่องมือทางการเงิน
- ทักษะการสื่อสารและการเขียนรายงาน

**การศึกษา:**
- ปริญญาตรี สาขาการบัญชี มหาวิทยาลัยห
